In [1]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 8.2 MB/s eta 0:00:00


In [3]:
!pip install category-encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 2.8 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap

In [6]:
!pip install shapash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 652.6/652.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.7/642.7 kB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 25.2 MB/s eta 0:00:00
  Created wheel for dash-renderer: filename=dash_renderer-1.8.3-py3-none-any.whl size=1013934 sha256=93dc06c2f657a425e66f86c41a21f4df148b39a5303d4dee0688654588efa60e
  Stored in directory: /root/.cache/pip/wheels/f7/4e/39/ef5c7473de15b5f1fc08576a98c3260c095ab162e750758ade
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669714 sha256=c90f25d177fd697cecfabfd86c34b012c098c83a7890d79d537e2

In [7]:
from shapash.data.data_loader import data_loading

In [8]:
titan_df, titan_dict = data_loading('titanic')
del titan_df['Name']

In [9]:
titan_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,,
1,0,Third class,male,22.0,1,0,7.25,Southampton,Mr
2,1,First class,female,38.0,1,0,71.28,Cherbourg,Mrs
3,1,Third class,female,26.0,0,0,7.92,Southampton,Miss
4,1,First class,female,35.0,1,0,53.10,Southampton,Mrs
5,0,Third class,male,35.0,0,0,8.05,Southampton,Mr


In [10]:
y = titan_df['Survived']
X = titan_df.drop('Survived', axis=1)

In [11]:
varcat=['Pclass','Sex','Embarked','Title']

In [12]:
categ_encoding = OrdinalEncoder(cols=varcat, \
                                handle_unknown='ignore', \
                                return_df=True).fit(X)
X = categ_encoding.transform(X)

In [13]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.75, random_state=1)

rf = RandomForestClassifier(n_estimators=100,min_samples_leaf=3)
rf.fit(Xtrain, ytrain)

RandomForestClassifier(min_samples_leaf=3)

In [14]:
ypred=pd.DataFrame(rf.predict(Xtest),columns=['pred'],index=Xtest.index)

# Compute Shapley Contributions with Shap

In [ ]:
explainer = shap.KernelExplainer(rf.predict_proba, Xtest)
shap_contrib = explainer.shap_values(Xtest)

# Use Shapash With Shapley Contributions

In [17]:
from shapash import SmartExplainer

In [20]:
xpl = SmartExplainer(
    model=rf,
    preprocessing=categ_encoding,
    features_dict=titan_dict)

In [24]:
xpl.compile(
    contributions=shap_contrib, # Shap Contributions pd.DataFrame
    y_pred=ypred,
    y_target=ytest, # Optional: allows to display True Values vs Predicted Values
    x=Xtest
)

In [27]:
xpl.plot.local_plot(index=85)

In [23]:
summary_df = xpl.to_pandas(max_contrib=3,positive=True,proba=True)
summary_df.head()

,pred,proba,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
863,1,0.809327,Sex,female,0.215846,Title of passenger,Mrs,0.136616,Ticket class,First class,0.135005
224,0,0.981119,Title of passenger,Mr,0.100721,Sex,male,0.07869,Passenger fare,7.9,0.067696
85,1,0.864417,Sex,female,0.19609,Title of passenger,Miss,0.171313,Ticket class,Second class,0.094955
681,1,0.695143,Title of passenger,Miss,0.155351,Port of embarkation,Queenstown,0.142937,Sex,female,0.140913
536,1,0.928014,Title of passenger,Miss,0.167905,Ticket class,Second class,0.13505,Sex,female,0.107437


In [30]:
xpl.plot.features_importance()

In [31]:
xpl.plot.contribution_plot(col='Pclass')

In [32]:
app = xpl.run_app(title_story='ACV backend')

Dash is running on http://0.0.0.0:8050/



INFO:dash.dash:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app 'shapash.webapp.smart_app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://172.28.0.12:8050
INFO:werkzeug:Press CTRL+C to quit


In [33]:
app.kill()